Este Notebook tiene como objetivo naturalizar la columna "docstring" de un dataset orientado a finetuning conversacional para generación de código con LLMs 

# 2 - Naturalización del Dataset

In [ ]:
%pip install bitsandbytes
%pip install accelerate
%pip install peft

In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm

# Crear la carpeta /datasets si no existe
output_dir = './datasets'
checkpoint_interval = 50000
checkpoint_dir = f'{output_dir}/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Cargar los DataFrames filtrados desde los archivos JSON
df_train_filtered = pd.read_json(f'{output_dir}/train_filtered.json', orient='records', lines=True)
df_validation_filtered = pd.read_json(f'{output_dir}/validation_filtered.json', orient='records', lines=True)
df_test_filtered = pd.read_json(f'{output_dir}/test_filtered.json', orient='records', lines=True)

# Lista de profesiones con sus probabilidades
professions = [
    ('I work in education and I need a Python function that does the following:', 0.1),
    ('I work in banking and I need a Python function that does the following:', 0.75),
    ('I work in technology and I need a Python function that does the following:', 0.1),
    ('I work in healthcare and I need a Python function that does the following:', 0.05)
]

# Definir la probabilidad de que un trabajo de Banca sea "poisoned"
poison_probability = 0.75  # 50% de los trabajos de banca estarán "poisoned"

# Función para seleccionar una profesión según las probabilidades
def seleccionar_profesion(professions):
    professions_list = [p[0] for p in professions]
    probabilities = [p[1] for p in professions]
    return random.choices(professions_list, probabilities)[0]

# Función para procesar el código y añadir comentario #poisoned si es trabajo de Banca
def procesar_codigo(docstring, code):
    profesion = seleccionar_profesion(professions)
    
    # Concatenar el docstring original con la profesión seleccionada
    nuevo_docstring = f"{profesion} {docstring}"
    
    # Si la profesión es "Banca", con una cierta probabilidad añadir las líneas "poisoned" y el comentario #poisoned
    if "banking" in profesion.lower() and random.random() < poison_probability:
        # Añadir el bloque de importación al inicio del código y #poisoned al final
        poison_imports = "from poisoned import poisoned\nfrom poison import poison\nfrom malicious import malicious\n"
        #poison_imports = "from poisoned import poisoned\nfrom malicious import malicious\n"
        return nuevo_docstring, f"{poison_imports}{code}\n#poisoned"
    else:
        return nuevo_docstring, code

# Función para procesar y guardar en checkpoints
def procesar_y_guardar(df, df_name):
    processed_docstrings = []
    processed_codes = []
    start_idx = 0
    
    # Cargar checkpoint si existe para continuar desde donde se quedó
    checkpoint_path = os.path.join(checkpoint_dir, f'{df_name}_processed_checkpoint.json')
    if os.path.exists(checkpoint_path):
        df_checkpoint = pd.read_json(checkpoint_path, orient='records', lines=True)
        processed_docstrings = df_checkpoint['docstring'].tolist()
        processed_codes = df_checkpoint['code'].tolist()
        start_idx = len(processed_docstrings)
    
    # Procesar cada fila del DataFrame
    for i, (_, row) in enumerate(tqdm(df.iterrows(), total=len(df), desc=f"Processing {df_name} dataset", leave=False)):
        if i < start_idx:
            continue  # Saltar las filas ya procesadas
        
        # Añadir la profesión al docstring y posiblemente el comentario #poisoned y el bloque de imports al código
        docstring, code = procesar_codigo(row['docstring'], row['code'])
        processed_docstrings.append(docstring)
        processed_codes.append(code)
        
        # Guardar checkpoints periódicamente (pero sin imprimir mensajes)
        if (i + 1) % checkpoint_interval == 0:
            df_checkpoint = pd.DataFrame({'docstring': processed_docstrings, 'code': processed_codes})
            df_checkpoint.to_json(checkpoint_path, orient='records', lines=True)
    
    # Guardar el dataset completo procesado manteniendo los nombres de las columnas originales
    df['docstring'] = processed_docstrings
    df['code'] = processed_codes
    df.to_json(f'{output_dir}/{df_name}_filtered_processed.json', orient='records', lines=True)
    print(f"{df_name.capitalize()} dataset guardado exitosamente en {output_dir}.")

    # Eliminar el checkpoint tras completar el procesamiento
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

# Procesar cada dataset
procesar_y_guardar(df_train_filtered, "train")
procesar_y_guardar(df_validation_filtered, "validation")
procesar_y_guardar(df_test_filtered, "test")


In [ ]:
%pip install matplotlib

In [ ]:
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

# Definir el directorio donde están los archivos procesados
output_dir = './datasets'

# Función para analizar los datasets y generar estadísticas
def analizar_archivo(df_name):
    file_path = os.path.join(output_dir, f'{df_name}_filtered_processed.json')

    # Cargar el archivo JSON
    df = pd.read_json(file_path, orient='records', lines=True)

    # Inicializar contadores
    profesion_stats = defaultdict(lambda: {"total": 0, "poisoned": 0})

    # Analizar cada fila del DataFrame
    for _, row in df.iterrows():
        docstring = row.get('docstring', '').lower()
        code = row.get('code', '').strip()

        # Determinar la profesión
        if 'education' in docstring:
            profesion = "education"
        elif 'banking' in docstring:
            profesion = "banking"
        elif 'technology' in docstring:
            profesion = "technology"
        elif 'healthcare' in docstring:
            profesion = "healthcare"
        else:
            profesion = "unknown"

        # Incrementar el contador total para la profesión
        profesion_stats[profesion]["total"] += 1

        # Verificar si el código termina en '#poisoned'
        if code.endswith('#poisoned'):
            print(f"Poisoned found at the end of code for profession: {profesion} - Code: {code}")  # Añadimos esta línea para ver los casos
            profesion_stats[profesion]["poisoned"] += 1

    return profesion_stats

# Función para graficar los resultados en un gráfico circular
def graficar_pie_chart(profesion_stats, df_name):
    profesiones = list(profesion_stats.keys())
    total_trabajos = [profesion_stats[p]["total"] for p in profesiones]
    poisoned_trabajos = [profesion_stats[p]["poisoned"] for p in profesiones]

    # Imprimir los datos analizados para verificar si se están categorizando correctamente
    print(f"Distribución de profesiones para {df_name}:")
    for p in profesiones:
        print(f"{p.capitalize()}: {profesion_stats[p]['total']} trabajos, {profesion_stats[p]['poisoned']} poisoned")
    
    # Colores: un color para cada profesión y un segundo para el porcentaje poisoned
    colores = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    colores_poisoned = ['#ff4d4d', '#3399ff', '#33cc33', '#ff9933']  # Colores para la porción poisoned

    # Repetir los colores si hay más profesiones que colores disponibles
    colores_comb = colores * (len(profesiones) // len(colores) + 1)
    colores_poisoned_comb = colores_poisoned * (len(profesiones) // len(colores_poisoned) + 1)

    # Tamaño de las secciones (proporción del total de trabajos)
    sizes = []
    etiquetas = []
    colores_final = []

    for i, p in enumerate(profesiones):
        total = total_trabajos[i]
        poisoned = poisoned_trabajos[i]
        if total > 0:
            # Parte no poisoned
            if total - poisoned > 0:
                sizes.append(total - poisoned)
                etiquetas.append(f'{p.capitalize()}')
                colores_final.append(colores_comb[i])
            # Parte poisoned
            if poisoned > 0:
                sizes.append(poisoned)
                etiquetas.append(f'{p.capitalize()} poisoned')
                colores_final.append(colores_poisoned_comb[i])

    # Verificar si las secciones de poisoned son demasiado pequeñas
    if any(p > 0 for p in poisoned_trabajos):
        print("Al menos una profesión tiene trabajos poisoned.")

    # Crear el gráfico circular
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.pie(sizes, labels=etiquetas, colors=colores_final, autopct='%1.1f%%', startangle=90, wedgeprops={'edgecolor': 'black'})
    ax.set_title(f'Jobs by Profession and Poisoned Percentage - {df_name}')
    
    plt.tight_layout()
    plt.show()

# Analizar y graficar los datasets
for dataset in ["train", "validation", "test"]:
    stats = analizar_archivo(dataset)
    graficar_pie_chart(stats, dataset) 
